In [17]:
import os
import subprocess as sp
import time

import numpy as np
from pymatgen.core.structure import Structure

root_dir = '/home/lonya/doc/deeph/CrI3'
pseudo_dir = os.path.join(root_dir, 'pseudo')
orbital_dir = os.path.join(root_dir, 'orbital')

def paral_str(is_paral):
    if is_paral:
        return "paral"
    return "nonparal"

os.chdir(root_dir)

In [18]:
np.random.seed(42)

structure = Structure.from_file(os.path.join(root_dir, 'CrI3.cif'))
structure_supercell = Structure(structure.lattice.matrix,
                                structure.atomic_numbers,
                                structure.frac_coords,
                                coords_are_cartesian=False,
                                to_unit_cell=True)
structure_supercell.make_supercell([[2, 0, 0], [0, 2, 0], [0, 0, 1]])

N_supercell = len(structure_supercell)

scf_thr = "1.0e-4"

kpt_file = fr"""K_POINTS
0
Gamma
3 3 3 0 0 0
"""

input_file_paral = fr"""INPUT_PARAMETERS
pseudo_dir              {pseudo_dir}
orbital_dir             {orbital_dir}
ntype                   2
nspin                   1
calculation             scf
basis_type              lcao
ecutwfc                 100
scf_thr                 {scf_thr}
scf_nmax                100
gamma_only              0

ks_solver               genelpa
smearing_method         gaussian
smearing_sigma          0.05

mixing_type             pulay
mixing_beta             0.4

out_mat_hs2             1
"""

input_file_nonparal = fr"""INPUT_PARAMETERS
pseudo_dir              {pseudo_dir}
orbital_dir             {orbital_dir}
ntype                   2
noncolin                1
lspinorb                0
calculation             scf
basis_type              lcao
ecutwfc                 100
scf_thr                 {scf_thr}
scf_nmax                100
gamma_only              0

ks_solver               genelpa
smearing_method         gaussian
smearing_sigma          0.05

mixing_type             pulay
mixing_beta             0.4

out_mat_hs2             1
"""

def gen_dft(dir, is_paral, structures, magmoms):
    os.makedirs(dir, exist_ok=True)
    for index_stru in range(1, structures + 1):
        origin_cart_coords = structure_supercell.cart_coords

        disorder_x = (np.random.rand(N_supercell, 1) - 0.5) * 0.2 # -0.1 ~ 0.1
        disorder_y = (np.random.rand(N_supercell, 1) - 0.5) * 0.2
        disorder_z = (np.random.rand(N_supercell, 1) - 0.5) * 0.2

        cart_coords = origin_cart_coords + np.concatenate([disorder_x, disorder_y, disorder_z], axis=-1)

        structure_supercell_perturb = Structure(structure_supercell.lattice.matrix,
                                                structure_supercell.atomic_numbers,
                                                cart_coords,
                                                coords_are_cartesian=True,
                                                to_unit_cell=True)
    
        numbers = structure_supercell_perturb.atomic_numbers
        NUM_ATOM = len(numbers)
        lattice = structure_supercell_perturb.lattice.matrix
        frac_coords = structure_supercell_perturb.frac_coords
        origin_frac_coords = structure_supercell.frac_coords
        Cr_frac_coords_str = ''
        I_frac_coords_str = ''
        
        for index_magmom in range(1, magmoms + 1):
            for i in range(NUM_ATOM):
                if origin_frac_coords[i, 2] > 0.24 and origin_frac_coords[i, 2] < 0.41:
                    if numbers[i] == 24:
                        Cr_frac_coords_str += f'{frac_coords[i, 0]:.16f} {frac_coords[i, 1]:.16f} {frac_coords[i, 2] + 0.166666:.16f} 0 0 0'
                        if is_paral == False:
                            Cr_frac_coords_str += f' mag {3.5 + 0.4 * np.random.rand()} angle1 {360.0 * np.random.rand()} angle2 {360.0 * np.random.rand()}\n'
                        else:
                            Cr_frac_coords_str += '\n'
                    elif numbers[i] == 53:
                        I_frac_coords_str += f'{frac_coords[i, 0]:.16f} {frac_coords[i, 1]:.16f} {frac_coords[i, 2] + 0.166666:.16f} 0 0 0'
                        if is_paral == False:
                            I_frac_coords_str += f' mag {-0.02 + 0.03 * np.random.rand()} angle1 {360.0 * np.random.rand()} angle2 {360.0 * np.random.rand()}\n'
                        else:
                            I_frac_coords_str += '\n'
                    else:
                        print(numbers[i])
                        raise ValueError("numbers[i] error")
            
            work_dir = os.path.join(dir, str(index_stru) + "_" + str(index_magmom))
            os.makedirs(work_dir, exist_ok=True)
            
            stru_file = fr"""ATOMIC_SPECIES
I 126.905 I_ONCV_PBE-1.0.upf
Cr 51.996 Cr_ONCV_PBE-1.0.upf

NUMERICAL_ORBITAL
I_gga_6au_100Ry_2s2p2d1f.orb
Cr_gga_7au_100Ry_4s2p2d1f.orb

LATTICE_CONSTANT
1.8897259886 # 1.8897259886 Bohr = 1.0 Angstrom

LATTICE_VECTORS
{lattice[0, 0]:.16f} {lattice[0, 1]:.16f} {lattice[0, 2]:.16f}
{lattice[1, 0]:.16f} {lattice[1, 1]:.16f} {lattice[1, 2]:.16f}
{lattice[2, 0]:.16f} {lattice[2, 1]:.16f} {lattice[2, 2]:.16f}

ATOMIC_POSITIONS
Direct
I
0.0
24
{I_frac_coords_str}

Cr
{3.7*int(is_paral)}
8
{Cr_frac_coords_str}
"""

            with open(os.path.join(work_dir, 'STRU'), 'w') as f:
                f.write(stru_file)

            with open(os.path.join(work_dir, 'KPT'), 'w') as f:
                f.write(kpt_file)

            with open(os.path.join(work_dir, 'INPUT'), 'w') as f:
                if is_paral:
                    f.write(input_file_paral)
                else:
                    f.write(input_file_nonparal)
                    
gen_dft(os.path.join(root_dir, "paral"), True, 100, 10)
gen_dft(os.path.join(root_dir, "nonparal"), False, 100, 10)

/root/miniconda3/lib/python3.11/site-packages/pymatgen/io/cif.py:1229: UserWarning: Issues encountered while parsing CIF: 4 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [ ]:
os.environ['OMP_NUM_THREADS'] = '8'

targets = ["1_1"]
is_paral = False

for target in targets:
    work_dir = os.path.join(root_dir, paral_str(is_paral), target)
    os.chdir(work_dir)
    sp.run(['mpirun', '-np', '1', 'abacus'])
    #sp.run(['mpirun', '-np', '1', 'abacus'], stdout=sp.DEVNULL, stderr=sp.DEVNULL)




                                                                                     
                              ABACUS v3.6.0

               Atomic-orbital Based Ab-initio Computation at UStc                    

                     Website: http://abacus.ustc.edu.cn/                             
               Documentation: https://abacus.deepmodeling.com/                       
                  Repository: https://github.com/abacusmodeling/abacus-develop       
                              https://github.com/deepmodeling/abacus-develop         
                      Commit: 059fc16 (Sat Apr 13 12:08:33 2024 +0800)

 Wed May 22 22:22:52 2024
 MAKE THE DIR         : OUT.ABACUS/
 RUNNING WITH DEVICE  : CPU / 13th Gen Intel(R) Core(TM) i9-13900K

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
 Pseudopotentials with additional electrons can yield (more) accurate outcomes, but may be less efficient.
 If you're confident th